In [12]:
!pip install PyPDF2


### Extraemos el texto de la pagina del pdf en la que están las notas de la PAU.

In [13]:
import requests
import PyPDF2
import io
import pandas as pd
import re

# URL del PDF
url = "https://www.universidades.gob.es/wp-content/uploads/2023/03/NOTA-PAU2022.pdf"

# Descarga el PDF
response = requests.get(url)
pdf_data = io.BytesIO(response.content)

# Crea un lector de PyPDF2
pdf_reader = PyPDF2.PdfReader(pdf_data)

# Ojo: las páginas empiezan en índice 0, así que la página 5
# corresponde al índice 4 (en caso de que el PDF no tenga portadas extras).
page_number = 4  

# Extrae el texto de la página deseada
page_text = pdf_reader.pages[page_number].extract_text()

# Muestra el texto
print(page_text)

    
5 de 7 
 
  
Por comunidad autónoma, l a nota media oscila entre el  6,3 en la UNED  (Estado)  y el 7,4 en  Castilla y 
León y la R ioja. El porcentaje de sobresalientes  se sitúa entre el 2,1% en la Comunitat Valenciana y el 
13,6 % en el Principado de Asturias.  
PAU  genérica. Fase específica . 
En la fase específica  de la PAU genérica se matriculan  275.551 estudiantes ( 221.993 en fase general y 
específica y 53.558 únicamente en fase específica). El 89,4% de los matriculados proceden de 
bachillerato, el 7,3% de F.P y enseñanzas artísticas.  
 
Por sexo, el 58,3% de los matriculados son mujeres. Esta cifra se sitúa en el  57,7% para los estudiantes 
procedentes de bachillerato y en el 66,5% para los e studiantes procedentes de FP y enseñanzas 
artísticas.  
 
 
 6,36,56,66,76,76,86,96,96,96,97,07,17,17,17,17,37,37,47,4
EstadoBalears (Illes)GaliciaCastilla-La ManchaComunitat ValencianaCanariasCataluñaMadrid (Comunidad de)AragónTotalExtremaduraMurcia (Región de)Navarra (Comun

### Acortamos el texto extraido eliminando lo que hay después de la informaición que nos interesa

In [14]:
# Definimos los marcadores para recortar el texto
start_marker = "5 de 7"
end_marker = "ordinaria"

# Buscamos el índice de inicio
start_index = page_text.find(start_marker)
if start_index == -1:
    raise ValueError(f"No se encontró el marcador de inicio '{start_marker}' en el texto.")

# Buscamos el índice de final (sumamos la longitud de end_marker para incluirlo)
end_index = page_text.find(end_marker, start_index)
if end_index == -1:
    raise ValueError(f"No se encontró el marcador de fin '{end_marker}' en el texto.")
end_index += len(end_marker)  # para incluir el marcador en el texto resultante

# Extraemos solo la parte deseada
extracted_text = page_text[start_index:end_index]

# Opcionalmente, podemos hacer un 'strip()' para limpiar espacios en extremos
extracted_text = extracted_text.strip()

print(extracted_text)

5 de 7 
 
  
Por comunidad autónoma, l a nota media oscila entre el  6,3 en la UNED  (Estado)  y el 7,4 en  Castilla y 
León y la R ioja. El porcentaje de sobresalientes  se sitúa entre el 2,1% en la Comunitat Valenciana y el 
13,6 % en el Principado de Asturias.  
PAU  genérica. Fase específica . 
En la fase específica  de la PAU genérica se matriculan  275.551 estudiantes ( 221.993 en fase general y 
específica y 53.558 únicamente en fase específica). El 89,4% de los matriculados proceden de 
bachillerato, el 7,3% de F.P y enseñanzas artísticas.  
 
Por sexo, el 58,3% de los matriculados son mujeres. Esta cifra se sitúa en el  57,7% para los estudiantes 
procedentes de bachillerato y en el 66,5% para los e studiantes procedentes de FP y enseñanzas 
artísticas.  
 
 
 6,36,56,66,76,76,86,96,96,96,97,07,17,17,17,17,37,37,47,4
EstadoBalears (Illes)GaliciaCastilla-La ManchaComunitat ValencianaCanariasCataluñaMadrid (Comunidad de)AragónTotalExtremaduraMurcia (Región de)Navarra (Comunidad 

# Seleccionamos la información que nos interesa (Nota media en EBAU  por Comunidades autónomas) y la formateamos como DataFrame

In [16]:
# --- 1) LOCALIZAR LÍNEAS DE INTERÉS ---

lines = extracted_text.splitlines()
line_notas = None
line_comunidades = None

for line in lines:
    # Buscar la línea con los valores de notas (sin la frase "Nota media...")
    if re.search(r'6,\d', line) and re.search(r'7,\d', line) and 'Nota media' not in line:
        line_notas = line.strip()
    # Buscar la línea que contenga todas las CCAA + "Castilla y León" + "Estado"
    if (
        "Estado" in line 
        and "Balears" in line 
        and "Galicia" in line 
        and "Castilla y León" in line
    ):
        line_comunidades = line.strip()

if line_notas is None or line_comunidades is None:
    raise ValueError("No se encontró la línea de notas o la línea de comunidades en el texto.")

# --- 2) Antes del split, cortamos donde empieza "Nota media" ---
# para que no se pegue a "Castilla y León"

# Si el substring "Nota media" aparece en la línea de comunidades, lo cortamos
index_of_nota_media = line_comunidades.find("Nota media")
if index_of_nota_media != -1:
    line_comunidades = line_comunidades[:index_of_nota_media]
    line_comunidades = line_comunidades.strip()

# --- 3) LIMPIAR Y EXTRAER LAS NOTAS ---
notas_str_list = re.findall(r"\d,\d", line_notas)
notas_float = [float(n.replace(',', '.')) for n in notas_str_list]

# --- 4) LIMPIAR Y EXTRAER LAS COMUNIDADES ---

# Usamos la estrategia del lookahead para separar cada una
regex_split = (
    r"(?="
    + r"|".join([
        "Estado",
        "Balears \\(Illes\\)",
        "Galicia",
        "Castilla-La Mancha",
        "Comunitat Valenciana",
        "Canarias",
        "Cataluña",
        "Madrid \\(Comunidad de\\)",
        "Aragón",
        "Total",
        "Extremadura",
        "Murcia \\(Región de\\)",
        "Navarra \\(Comunidad Foral de\\)",
        "País Vasco",
        "Andalucía",
        "Cantabria",
        "Asturias \\(Principado de\\)",
        "Rioja \\(La\\)",
        "Castilla y León"
    ])
    + r")"
)

comunidades_list = re.split(regex_split, line_comunidades)
comunidades_list = [c.strip() for c in comunidades_list if c.strip()]

# Comprobamos que ahora haya 19
if len(comunidades_list) != 19:
    raise ValueError(
        f"Se esperaban 19 comunidades (incluyendo 'Total'), pero se obtuvieron {len(comunidades_list)}"
    )

# --- 5) CREAR EL DATAFRAME ---
df = pd.DataFrame({
    "Comunidad Autónoma": comunidades_list,
    "Nota media": notas_float
})

df


,Comunidad Autónoma,Nota media
0,Estado,6.3
1,Balears (Illes),6.5
2,Galicia,6.6
3,Castilla-La Mancha,6.7
4,Comunitat Valenciana,6.7
5,Canarias,6.8
6,Cataluña,6.9
7,Madrid (Comunidad de),6.9
8,Aragón,6.9
9,Total,6.9
